In [1]:
API_KEY = api_key


In [2]:
def API_func(API_KEY, origin_excel_route, final_excel_route):
    #all variables:
    #API_KEY: chatgpt api key
    #origin_excel_route: route of target 1q1a excel / 從chatgpt生成的1q1a的excel
    #final_excel_route: route of final multiple-q to 1a excel / 最後想要生成的excel file存取位置+名字
    
    #all functions
    def findlist(target):
        start = target.rfind('[')
        end = target.rfind(']')
        if (start == -1) | (end == -1):
            return ""
        return target[start:end+1]
    
    #To check if the answer is made by multiple lists   Process starts from keyword "list": separate each list if number of list > 2
    #看看答案是否是多個list組成的   從list開始篩選：如果list後面接著的list多於2個：就切割
    def test_result(target):
        point = target.rfind('list')
        target = target[point:]
        try:
            start = target.rfind('[')
            if(start == -1):
                return False
            _next = target[:start]
            try: 
                start = _next.rfind('[')
                if(start == -1):
                    return False
                _next_ = _next[:start]
                return True
            except BaseException:
                return False
        except BaseException:
            return False
    
    #test if each question set has count < 5. If so, return false
    #測試是否所有問題集都短於5個問題。如果是：回傳false
    def test_simplifiedq(newq,limit):
        quest = []
        for x in newq:
            if len(x) > limit:
                return True
        return False

    def split_quest(x):
        q_list = eval(findlist(x))
        target = []
        for quest in q_list:
            if len(quest.split('？')) == 1:
                if len(quest.split('?')) == 1:
                    target.append(quest)
                else:
                    for each in quest.split('?'):
                        if len(each) == 0:
                            continue
                        target.append(each)
            else:
                for each in quest.split('？'): #分開一些chatgpt把問題全部縮進一個string的可能性，以？做區分 **注意？需要是中文的？
                    if len(each) == 0:
                        continue
                    target.append(each)
        return target

    
    #import all libraries:
    import pandas as pd
    import math
    import time
    import os
    import openai
    openai.api_key = API_KEY
    
    #read excel as origin_df / 讀取excel as origin_df
    origin_df = pd.read_excel(origin_excel_route)
    origin_df = origin_df.drop("Unnamed: 0", axis = 1)
    origin_df = origin_df.drop("人工查看列表", axis = 1)
    new_df = origin_df            #make copy of origin_df to prevent changes / make copy以免改到不該改的東西
    
    unprocess_q = []
    for x in range(len(new_df['answers'])):
        try:
            completion = openai.ChatCompletion.create(
                model="gpt-3.5-turbo",
                messages=[
                    {"role": "system", "content": "用繁體中文回答問題。"},
                    {"role": "user", "content": " 接下來我會給妳一段文字和一組問題，將那段文字視為那組問題的答案。請基於那段答案，請生成問題裡沒有包括但答案可以回答的問題。然後，評估你所生成的問題組中問題的語意相似度並依照相似度分組。接著，依照你所分出的組別，在保證內容不變的情況下，將每組問題整理概括成一個問句。最後，將你所給出的組別總結分為一個新組並將這個組內的整理問句寫成一個由string組成的python list。"},
                    {"role": "assistant", "content": f"答案：{new_df['answers'][x]}"},
                    {"role": "assistant", "content": f"問題：{new_df['questions'][x]}"}
                ],
                temperature = 0.2
            )
            #unprocess_newq: direct response from chatgpt / unprocess_newq: chatgpt直接輸出的答案，不做任何修改
            unprocess_q.append(completion.choices[0].message['content'])
        except BaseException:
            unprocess_q.append([])
            
    _q = []
    for x in unprocess_q:
        try:
            target = split_quest(x)
            _q.append(target)
        except BaseException:
            _q.append([])
    
    unprocess_qs = {}
    for x in range(len(_q)):
        if len(_q[x]) == 0:
            try:
                completion = openai.ChatCompletion.create(
                model="gpt-3.5-turbo",
                messages=[
                    {"role": "system", "content": "用繁體中文回答問題。"},
                    {"role": "user", "content": " 接下來我會給妳一段文字和一組問題，將那段文字視為那組問題的答案。請基於那段答案，請生成問題裡沒有包括但答案可以回答的問題。然後，評估你所生成的問題組中問題的語意相似度並依照相似度分組。接著，依照你所分出的組別，在保證內容不變的情況下，將每組問題整理概括成一個問句。最後，將你所給出的組別總結分為一個新組並將這個組內的整理問句寫成一個由string組成的python list。"},
                    {"role": "assistant", "content": f"答案：{new_df['answers'][x]}"},
                    {"role": "assistant", "content": f"問題：{new_df['questions'][x]}"}
                ],
                temperature = 0.2
            )
                unprocess_qs[x] = completion.choices[0].message['content']
            except BaseException:
                unprocess_qs[x] = []

    for x in list(unprocess_qs.keys()):
        try:
            target = split_quest(unprocess_qs[x])
            _q[x] = target
        except BaseException:
            _q[x] = _q[x]    
    
    import openpyxl
    wb = openpyxl.load_workbook(origin_excel_route)
    ws = wb.active

    curr = 0
    prev_count = 1
    indexes = range(2,len(_q)+1)
    for ind in range(len(indexes)-1):
        if len(_q[ind]) == 1:
            continue
        curr = indexes[ind] + prev_count

        for each in _q[ind]:
            ws.insert_rows(curr,1)
            ws[f'B{curr}'] = each
            curr += 1

        merge_ans = f"C{indexes[ind] + prev_count-1}:C{curr-1}"
        ws.merge_cells(merge_ans)

        merge_ind = f"A{indexes[ind] + prev_count-1}:A{curr-1}"
        ws.merge_cells(merge_ind)

        prev_count = curr - indexes[ind]
    
    file = wb.save(final_excel_route)
    
    return file
    

In [3]:
target_excel = ''
output_excel = ''
file = API_func(API_KEY, target_excel, output_excel)